<a href="https://colab.research.google.com/github/lorifin/PAL/blob/main/resume_actions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ✅ Installer nécessaire
!pip install -U gradio

import gradio as gr
import re

# ✅ Fonction de prétraitement améliorée
def preprocess_text(text):
    text = text.replace('\n', ' ').strip()
    # Simplifier des structures passives en infinitif
    text = re.sub(r'\bLa maquette doit être validée\b', 'Valider la maquette', text, flags=re.IGNORECASE)
    # Séparer les actions liées par "et", "puis"
    text = re.sub(r'\bet\b', '. ', text, flags=re.IGNORECASE)
    text = re.sub(r'\bpuis\b', '. ', text, flags=re.IGNORECASE)
    return text

# ✅ Fonction d'extraction de tâches robuste
def extract_tasks(text):
    text = preprocess_text(text)
    sentences = re.split(r'[.\n]', text)
    action_verbs = [
        "valider", "tester", "finaliser", "confirmer", "envoyer", "préparer", "planifier",
        "organiser", "relancer", "contacter", "rédiger", "mettre", "terminer", "vérifier",
        "réviser", "effectuer", "appeler", "répondre", "communiquer", "participer"
    ]
    tasks = ""
    for s in sentences:
        s_clean = s.strip()
        if len(s_clean) < 6:
            continue
        # Détecter verbe d'action OU participe passé courant
        if any(verb in s_clean.lower() for verb in action_verbs) or re.search(r'\b(\w+é[e]?[s]?)\b', s_clean.lower()):
            s_clean = s_clean[0].upper() + s_clean[1:]
            if not s_clean.endswith('.'):
                s_clean += '.'
            tasks += f"- [ ] {s_clean}\n"
            # Correction automatique des formes passives résiduelles
            tasks = re.sub(r'- \[ \] Le plan de com finalisé\.', '- [ ] Finaliser le plan de communication.', tasks)

    if not tasks.strip():
        tasks = "⚠️ Aucune tâche détectée. Fournissez un texte contenant des actions."

    return tasks



# ✅ Interface Gradio
gr.Interface(
    fn=extract_tasks,
    inputs=gr.Textbox(lines=12, placeholder="Collez ici un mail ou CR"),
    outputs="textbox",
    title="✅ Extracteur IA de tâches robuste",
    description="Générez une liste de tâches prête à coller dans Notion, Slack ou Trello depuis vos mails ou CR."
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cfaceae7e7c7cb17fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
